In [46]:
import pandas as pd
import ast

In [7]:
def lower(s):
    return s.lower()

In [177]:
import re
def strip(s):
    s=' '.join(s.split())
    return re.sub(r'[^A-Za-z0-9 ]+', '', s).lower()

In [31]:
j=pd.read_csv("jstor.csv")

In [32]:
c=pd.read_csv("comb.csv")

In [33]:
c['text']=c['text'].fillna("").apply(lower)

In [97]:
df=c.append(j)

C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\659228089.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=c.append(j)


In [99]:
df=df.drop_duplicates(["title"])

In [157]:
def conv(s):
    try:
        return ast.literal_eval(s)
    except:
        return s

In [203]:
df=df.set_index("title")

In [208]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [241]:
vectorizer = CountVectorizer(stop_words='english')


In [242]:
temp=df[df["abstract"]!=""]

In [243]:
X = vectorizer.fit_transform(temp["abstract"])


In [244]:
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())


In [245]:
df_bow_sklearn["title"]=temp.index


In [246]:
tfidf=df_bow_sklearn.set_index("title")

In [239]:
tfidf.to_csv("counts_abs.csv")

In [ ]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [249]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = list(df["abstract"])
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['years', 'since', 'japan', 'experienced', 'war', 'world', 'war', 'ii', 'government', 'japan', 'goj', 'proclaimed', 'new', 'japanese', 'constitution', 'drafted', 'united', 'states', 'general', 'headquarters', 'enforced', 'article', 'new', 'constitution', 'renouncing', 'use', 'force', 'international', 'disputes', 'contributed']


In [250]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 3), (26, 1), (27, 1), (28, 1), (29, 1)]


In [256]:
df["abstract"]

title
The Development of Japan Special Operations Command’s Capability: 21<sup>st</sup> Century Samurai Diplomats                                 it has been 71 years since japan experienced w...
@PhilosTEI:                                                                                                                                 the main objective of the clarinnl project  ph...
Neither High-Church, Low-Church, nor No-Church: Religious Dissatisfaction and Dissent in Bleak House                                        church bleak house esther religious anglican c...
Searching for Dr. Johnson:                                                                                                                  as you enter the rare books and music reading ...
“RESEARCH BASED ADVOCACY FOR POLICY CHANGE: Ban or Restrict & Discourage Mercury Amalgam Use to Safeguard Children’s Health in Pakistan”    mercury is naturally occurring element in envi...
                                            

In [252]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(tfidf, num_topics=3, id2word = dictionary, passes=50)

NameError: name 'doc_term_matrix' is not defined

In [251]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"digital" + 0.004*"new" + 0.004*"internet" + 0.003*"article" + '
  '0.003*"research" + 0.003*"public" + 0.003*"google" + 0.002*"books" + '
  '0.002*"humanities" + 0.002*"data"'),
 (1,
  '0.004*"digital" + 0.004*"de" + 0.003*"new" + 0.003*"corpus" + 0.003*"data" '
  '+ 0.003*"analysis" + 0.003*"also" + 0.003*"cultural" + 0.003*"la" + '
  '0.002*"books"'),
 (2,
  '0.007*"digital" + 0.006*"research" + 0.004*"social" + 0.004*"humanities" + '
  '0.003*"de" + 0.003*"use" + 0.003*"new" + 0.003*"data" + 0.003*"language" + '
  '0.003*"cultural"'),
 (3,
  '0.004*"digital" + 0.004*"de" + 0.003*"library" + 0.003*"web" + 0.003*"work" '
  '+ 0.003*"american" + 0.002*"data" + 0.002*"la" + 0.002*"internet" + '
  '0.002*"access"'),
 (4,
  '0.004*"library" + 0.004*"reading" + 0.004*"students" + 0.003*"digital" + '
  '0.003*"research" + 0.003*"study" + 0.003*"english" + 0.003*"de" + '
  '0.003*"books" + 0.003*"web"'),
 (5,
  '0.007*"de" + 0.005*"digital" + 0.004*"research" + 0.004*"social" 

In [263]:
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, 
                              corpus=corpus, 
                              dictionary=id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\lucia\miniconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [260]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

C:\Users\lucia\miniconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


FileNotFoundError: [Errno 2] No such file or directory: './results/ldavis_prepared_10'

In [159]:
df["text"]=df["text"].apply(conv)

C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\3611968549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"]=df["text"].apply(conv)


In [172]:
text=[]
for t in df["text"]:
    if isinstance(t,str):
        text.append(t)
    else:
        temp=""
        for k in t:
            for i in range(0,t[k]):
                temp+=k
                temp+=" "
        text.append(temp)

In [179]:
df["text"]=df["text"].apply(strip)

C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\2521916661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"]=df["text"].apply(strip)


In [182]:
df["abstract"]=df["abstract"].fillna("").apply(strip)

C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\1023429554.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["abstract"]=df["abstract"].fillna("").apply(strip)


In [198]:
df.to_csv("full.csv")

In [44]:
import ast

In [45]:
df["journal"].value_counts().head(20)

Behavioral and Brain Sciences                 151
Benjamins E-Books                             112
SAGE KNOWLEDGE                                 76
Victorian Literature and Culture               62
American Libraries                             56
Elgar E-Books                                  51
Victorian Periodicals Review                   37
portal: Libraries and the Academy              34
Victorian Review                               33
Early American Literature                      32
Library Hi Tech                                32
Berkeley Technology Law Journal                31
Book History                                   28
The Moving Image                               28
PMLA                                           28
Library Trends                                 28
MELUS: Multi-Ethnic Literature of the U.S.     27
Interlending & Document Supply                 27
Library Hi Tech News                           27
Journal of Cultural Analytics                  26


In [150]:
df.to_csv("complete.csv")

In [17]:
a=pd.read_csv("arxiv.csv")

In [18]:
d=pd.read_csv("dhq.csv")

In [19]:
o=pd.read_csv("open_source.csv")
c=pd.read_csv("jca.csv")

In [20]:
c['text']=c['text'].fillna("").apply(lower)

In [21]:
c=c[c['text'].str.contains('google|hathi|internet archive|gutenberg')]

In [22]:
d['text']=d['text'].fillna("").apply(lower)
d=d[d['text'].str.contains('google|hathi|internet archive|gutenberg')]

In [23]:
comb=j.append(o).append(c).append(a).append(d)

C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\1349516320.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb=j.append(o).append(c).append(a).append(d)
C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\1349516320.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb=j.append(o).append(c).append(a).append(d)
C:\Users\lucia\AppData\Local\Temp\ipykernel_4276\1349516320.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comb=j.append(o).append(c).append(a).append(d)


In [24]:
comb=comb.drop_duplicates(["title"])

In [25]:
comb["text"]=comb["text"].fillna("")

In [29]:
comb.to_csv("comb.csv")

In [30]:
comb

,Unnamed: 0,date,title,tag,author,abstract,text,url,journal,Unnamed: 0.1,match
0,0,2017-05-24,The Development of Japan Special Operations Co...,"['Political science - Military science', 'Phil...",['Ryota Akiba'],It has been 71 years since Japan experienced w...,The Development of Japan Special Operations Co...,http://www.jstor.org/stable/resrep13997,Daniel K. Inouye Asia-Pacific Center for Secur...,NaN,NaN
1,1,2017-01-01,@PhilosTEI:,['Information science - Informetrics'],[],The main objective of the CLARIN-NL project @ ...,UP 033 odijk odijkprinter 20171215 1557 Page 3...,http://www.jstor.org/stable/j.ctv3t5qjk.39,CLARIN in the Low Countries,NaN,NaN
2,2,2018-01-01,"Neither High-Church, Low-Church, nor No-Church...","['Arts - Literature', 'History - Historical me...",['Christian Dickinson'],"['church', 'bleak house', 'esther', 'religious...",105325dickstudannu4920349 Dickens Studies Annu...,ark://27927/phzjr66v4k5,Dickens Studies Annual: Essays on Victorian Fi...,NaN,NaN
3,3,2018-01-01,Searching for Dr. Johnson:,['Information science - Informetrics'],[],As you enter the Rare Books and Music Reading ...,Part 3 Archival Limits UN chapter Searching ...,http://www.jstor.org/stable/10.1163/j.ctvbqs8w9.8,Travelling Chronicles: News and Newspapers fro...,NaN,NaN
4,4,2018-09-01,“RESEARCH BASED ADVOCACY FOR POLICY CHANGE: Ba...,['Business - Industry'],"['Shiza Durrani', 'Hafsa Bashir', 'Mahmood A. ...",Mercury is naturally occurring element in envi...,EXECUTIVE SUMMARY This report describes and di...,http://www.jstor.org/stable/resrep24369,Sustainable Development Policy Institute,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
663,663,2016 10.1,Word Processor Art How Userfriendly Inhibits C...,NaN,['Rene Farrar'],My paper contains original Word Processor Art ...,nietzsche writes that our writing tools are al...,http://www.digitalhumanities.org//dhq/vol/10/1...,NaN,NaN,NaN
664,664,2009 3.2,Words Patterns and Documents Experiments in Ma...,NaN,"['Shlomo Argamon', 'Mark Olsen']",This introduces the set of papers reflecting i...,one of the emerging grand challenges for digit...,http://www.digitalhumanities.org//dhq/vol/3/2/...,NaN,NaN,NaN
666,666,2018 12.1,The Writeprints of Man a Stylometric Study of ...,NaN,"['Richard Forsyth', 'David Holmes']",Thomas Paines Rights of Man Part I 1791 Part I...,more than the writings of any other writer tho...,http://www.digitalhumanities.org//dhq/vol/12/1...,NaN,NaN,NaN
668,668,2011 5.3,Writing to be Found and Writing Readers,NaN,['John Cayley'],Poetic writing for programmable and network me...,in order to begin to write this essay i set ou...,http://www.digitalhumanities.org//dhq/vol/5/3/...,NaN,NaN,NaN
